In [ ]:
import re
import os
import torch
import glob
import hashlib

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from more_itertools import chunked


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
raw_data_folder = "data/cleaned"
model_name = "intfloat/multilingual-e5-base"
persist_directory = "chroma_db"

## READ_FILES

In [2]:
txt_files = glob.glob(os.path.join(raw_data_folder, "*.txt"))
md_files = glob.glob(os.path.join(raw_data_folder, "*.md"))

def read_txt_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    
def clean_text(text):
    #text.to_lower()  # Chuyển đổi chữ hoa thành chữ thường
    #text = re.sub(r'#* ', '', text)  # Remove Markdown headers
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = text.strip() 
    return text

markdown_text = ""

for file_path in md_files:
    try:
        content = read_txt_file(file_path)
        #content = clean_text(content)
        if content:
            markdown_text += f"\n\n## {os.path.basename(file_path)}\n\n{content}"
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

In [3]:
loeader = DirectoryLoader(raw_data_folder,
                           glob="**/*.md",
                             show_progress=True,
                              loader_cls=TextLoader,
                              loader_kwargs={"encoding": "utf-8"}
                              )
documents = loeader.load()
documents


100%|██████████| 1/1 [00:00<00:00, 997.93it/s]


[Document(metadata={'source': 'data\\cleaned\\page12.md'}, page_content='# Chương 1 Ma trận\n\n## 1.1 Các khái niệm cơ bản\n\nĐịnh nghĩa 1.1 (Ma trận) . Ma trận cỡ m × n là một bảng số (thực hoặc phức) hình chữ nhật có m hàng và n cột.\n\n$$ A = \\left( \\begin{matrix} { a }_{ 11 } & \\cdots & { a }_{ 1j } & \\cdots & { a }_{ 1n } \\\\ \\vdots & \\ddots & \\vdots & \\ddots & \\vdots \\\\ { a }_{ i1 } & \\cdots & { a }_{ ij } & \\cdots & { a }_{ in } \\\\ \\vdots & \\ddots & \\vdots & \\ddots & \\vdots \\\\ { a }_{ m1 } & \\cdots & { a }_{ mj } & \\cdots & { a }_{ mn } \\end{matrix} \\right) $$\n\nVí dụ 1.1\n\n$$ A = \\left( \\begin{matrix} 3 & 4 & 1 \\\\ 2 & 0 & 5 \\end{matrix} \\right)_{ 2 \\times 3 } , \\quad B = \\left( \\begin{matrix} 1 + i & 2 \\\\ 3-i & 4i \\end{matrix} \\right) $$\n\nA là ma trận cỡ 2 × 3 có 2 hàng và 3 cột. Các phần tử của ma trận A:\n\na11 = 3, a12 = 4, a13 = 1, a21 = 2, a22 = 0, a32 = 5.\n\nB là ma trận cỡ 2 × 2 có các phần tử trong phức.\n\nGhi chú\n\n• Ma t

## CHUCKING

In [4]:
headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
            ("####", "Header 4"),
            ]

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on = headers_to_split_on
)



# Split the documents into chunks based on the headers

all_chunks = []
for doc in documents:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        chunk.metadata["source"] = doc.metadata.get("source", "")
    all_chunks.extend(chunks)

#chunks = splitter.split_text(all_chunks)


In [5]:
all_chunks

[Document(metadata={'Header 1': 'Chương 1 Ma trận', 'Header 2': '1.1 Các khái niệm cơ bản', 'source': 'data\\cleaned\\page12.md'}, page_content='Định nghĩa 1.1 (Ma trận) . Ma trận cỡ m × n là một bảng số (thực hoặc phức) hình chữ nhật có m hàng và n cột.  \n$$ A = \\left( \\begin{matrix} { a }_{ 11 } & \\cdots & { a }_{ 1j } & \\cdots & { a }_{ 1n } \\\\ \\vdots & \\ddots & \\vdots & \\ddots & \\vdots \\\\ { a }_{ i1 } & \\cdots & { a }_{ ij } & \\cdots & { a }_{ in } \\\\ \\vdots & \\ddots & \\vdots & \\ddots & \\vdots \\\\ { a }_{ m1 } & \\cdots & { a }_{ mj } & \\cdots & { a }_{ mn } \\end{matrix} \\right) $$  \nVí dụ 1.1  \n$$ A = \\left( \\begin{matrix} 3 & 4 & 1 \\\\ 2 & 0 & 5 \\end{matrix} \\right)_{ 2 \\times 3 } , \\quad B = \\left( \\begin{matrix} 1 + i & 2 \\\\ 3-i & 4i \\end{matrix} \\right) $$  \nA là ma trận cỡ 2 × 3 có 2 hàng và 3 cột. Các phần tử của ma trận A:  \na11 = 3, a12 = 4, a13 = 1, a21 = 2, a22 = 0, a32 = 5.  \nB là ma trận cỡ 2 × 2 có các phần tử trong phức.  

## EMBEDING

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True}
)


if os.path.exists(persist_directory):
    print(f"Persist directory {persist_directory} already exists. Loading existing vectorstore.")
    vectorstore = Chroma(persist_directory=persist_directory,
                            embedding=embedding_model,
                          collection_name="math_vectors")
    #vectorstore.add_documents(all_chunks)
    BATCH_SIZE = 100
    for i, chunk in enumerate(chunked(all_chunks, BATCH_SIZE)):
        print(f"Processing chunk {i + 1}/{len(all_chunks) // BATCH_SIZE + 1}")
        vectorstore.add_documents(chunk)
else:
    vectorstore = Chroma.from_documents(
        documents=all_chunks,
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_name="math_vectors"
    )




#vectorstore.persist()

C:\Users\dongh\AppData\Local\Temp\ipykernel_16464\3186819850.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


Test

In [14]:
from langchain.chains import RetrievalQA


vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model,
    collection_name="math_vectors"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Nhập câu hỏi
query = "Ma trận là gì"

# Truy xuất các đoạn văn liên quan
docs = retriever.get_relevant_documents(query)

for i, doc in enumerate(docs):
    print(f"\n--- Kết quả {i+1} ---")
    print("Từ file:", doc.metadata.get("source", ""))
    print("header1", doc.metadata.get("Header 1", ""))
    print("header2", doc.metadata.get("Header 2", ""))
    print("header3", doc.metadata.get("Header 3", ""))
    print(doc.page_content, "...")


--- Kết quả 1 ---
Từ file: data\cleaned\page12.md
header1 Chương 1 Ma trận
header2 1.1 Các khái niệm cơ bản
header3 
Định nghĩa 1.1 (Ma trận) . Ma trận cỡ m × n là một bảng số (thực hoặc phức) hình chữ nhật có m hàng và n cột.  
$$ A = \left( \begin{matrix} { a }_{ 11 } & \cdots & { a }_{ 1j } & \cdots & { a }_{ 1n } \\ \vdots & \ddots & \vdots & \ddots & \vdots \\ { a }_{ i1 } & \cdots & { a }_{ ij } & \cdots & { a }_{ in } \\ \vdots & \ddots & \vdots & \ddots & \vdots \\ { a }_{ m1 } & \cdots & { a }_{ mj } & \cdots & { a }_{ mn } \end{matrix} \right) $$  
Ví dụ 1.1  
$$ A = \left( \begin{matrix} 3 & 4 & 1 \\ 2 & 0 & 5 \end{matrix} \right)_{ 2 \times 3 } , \quad B = \left( \begin{matrix} 1 + i & 2 \\ 3-i & 4i \end{matrix} \right) $$  
A là ma trận cỡ 2 × 3 có 2 hàng và 3 cột. Các phần tử của ma trận A:  
a11 = 3, a12 = 4, a13 = 1, a21 = 2, a22 = 0, a32 = 5.  
B là ma trận cỡ 2 × 2 có các phần tử trong phức.  
Ghi chú  
• Ma trận A cỡ m × n thường được ký hiệu bởi A = (aij)m×n.  
• T

In [ ]:
# Load model
model = SentenceTransformer(model_name, device=DEVICE)

In [ ]:
# Tạo embedding
texts = [chunk.page_content for chunk in chunks]
embeddings = model.encode(texts, 
                          convert_to_tensor=False,
                          normalize_embeddings=True)

In [10]:
embeddings

array([[ 0.01175354,  0.04333695,  0.01675817, ..., -0.01774279,
        -0.02854558,  0.07406582],
       [ 0.01095208,  0.0504429 , -0.01873817, ..., -0.02134096,
        -0.03130301,  0.03099393]], dtype=float32)